In [2]:
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
vas_data_dir = os.path.join("Video2ImageDataset_noFace")

In [11]:
len(os.listdir("Video2ImageDataset_noFace/fake"))

26460

In [10]:
len(os.listdir("Video2ImageDataset_noFace/real"))

2812

In [21]:
%pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [23]:
import splitfolders

splitfolders.ratio('Video2ImageDataset_noFace', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 29272 files [00:14, 1964.35 files/s]


In [25]:
len(os.listdir("output/train/real")), len(os.listdir("output/train/fake"))

(2249, 21168)

In [26]:
len(os.listdir("output/test/real")), len(os.listdir("output/test/fake"))

(282, 2647)

In [27]:
len(os.listdir("output/val/real")), len(os.listdir("output/val/fake"))

(281, 2646)

In [29]:
vas_train_data_path = "output/train"

In [37]:
vas_train_data_gen = ImageDataGenerator(rescale = 1./255)

target_size = (224, 224) 
batch_size = 100

vas_train_data_generator = vas_train_data_gen.flow_from_directory(
    vas_train_data_path,
    target_size= target_size, 
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=True
)

Found 23417 images belonging to 2 classes.


In [38]:
vas_train_data_generator.class_indices

{'fake': 0, 'real': 1}

In [33]:
%ls output/train/

fake/ real/


In [36]:
%rm -rf output/train/.ipynb_checkpoints

In [39]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam  

def build_model():
    shape = (224, 224, 3)
    
    Xception_mod_trained = Xception(input_shape=shape, include_top=False, weights='imagenet')  
    
    
    final_model_transfer = Sequential()
    
    final_model_transfer.add(Xception_mod_trained)
    
    final_model_transfer.add(layers.GlobalAveragePooling2D())
    final_model_transfer.add(layers.Dense(1024, activation='relu'))
    final_model_transfer.add(layers.Dropout(0.5))
    
    final_model_transfer.add(layers.Dense(1, activation='sigmoid'))  
    
    final_model_transfer.compile(optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'], loss = 'binary_crossentropy')


    return final_model_transfer


model_trans = build_model()

In [41]:
epochs = 10
history = model_trans.fit(vas_train_data_generator, epochs=epochs)

Epoch 1/10
235/235 [==============================] - 2825s 12s/step - loss: 0.3464 - accuracy: 0.8984
Epoch 2/10
235/235 [==============================] - 2910s 12s/step - loss: 0.2973 - accuracy: 0.9040
Epoch 3/10
235/235 [==============================] - 3506s 15s/step - loss: 0.2731 - accuracy: 0.9040
Epoch 4/10
235/235 [==============================] - 2833s 12s/step - loss: 0.2441 - accuracy: 0.9049
Epoch 5/10
235/235 [==============================] - 2857s 12s/step - loss: 0.2115 - accuracy: 0.9090
Epoch 6/10
235/235 [==============================] - 2831s 12s/step - loss: 0.1781 - accuracy: 0.9236
Epoch 7/10
235/235 [==============================] - 2802s 12s/step - loss: 0.1446 - accuracy: 0.9392
Epoch 8/10
235/235 [==============================] - 2790s 12s/step - loss: 0.1112 - accuracy: 0.9545
Epoch 9/10
235/235 [==============================] - 2808s 12s/step - loss: 0.0834 - accuracy: 0.9668
Epoch 10/10
235/235 [==============================] - 2784s 12s/step - l

In [49]:
vas_test_data_path = "output/test"

vas_test_data_gen = ImageDataGenerator(rescale = 1./255)

target_size = (224, 224) 
batch_size = 100

vas_test_data_generator = vas_test_data_gen.flow_from_directory(
    vas_test_data_path,
    target_size= target_size, 
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=True
)

Found 2929 images belonging to 2 classes.


In [50]:
vas_test_data_generator.class_indices

{'fake': 0, 'real': 1}

In [46]:
%ls output/test/

fake/ real/


In [47]:
%rm -rf output/test/.ipynb_checkpoints

In [51]:
model_trans.evaluate(vas_test_data_generator)

30/30 [==============================] - 78s 3s/step - loss: 0.2754 - accuracy: 0.9249


[0.27537328004837036, 0.9248890280723572]

In [53]:
model_trans.save("vas_transfer_model.keras")